# Import relevant libraries

In [18]:
import os
import json
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas import GeoDataFrame
from pyproj import CRS
from matplotlib import rc
import folium
from folium.plugins import MarkerCluster
from folium import plugins
import shapely
from shapely.geometry import Polygon
from pyproj import CRS
from shapely.geometry import Point

# Load UK shape file

In [19]:
uk = gpd.read_file("London_Borough_Excluding_MHW.json")
crs_27700 = CRS("EPSG:27700")

uk.crs = crs_27700
uk = uk[['NAME', 'geometry']]
uk = uk.to_crs("EPSG:4326")

uk = uk[['NAME', 'geometry']]
uk.rename(columns = {'NAME': 'district'}, inplace = True)

In [20]:
# set file path
base_path = '../..'
file_path = os.path.join(base_path, 'datasets', 'tech_roundabout_coordinates.csv')

# load data
tech_roundabout = pd.read_csv(file_path)

In [21]:
# set file path
base_path = '../..'
file_path = os.path.join(base_path, 'datasets', 'industry_added_cleaned_data.csv')

# load data
iac = pd.read_csv(file_path)

In [22]:
# set file path
base_path = '../..'
file_path = os.path.join(base_path, 'datasets', 'tech_industry_added_cleaned_data.csv')

# load data
tech_industries = pd.read_csv(file_path)

In [23]:
# set-up coordinates
tech_roundabout_coordinates = tech_roundabout[['CompanyName', 'latitude', 'longitude']]

tech_roundabout_coordinates = gpd.GeoDataFrame(
    tech_roundabout_coordinates.drop(['latitude','longitude'], axis = 1),
    geometry=[Point(xy) for xy in zip(tech_roundabout_coordinates.longitude,
                                      tech_roundabout_coordinates.latitude)])

In [24]:
# project to epsg:4326 --  = crs_27700
tech_roundabout_coordinates.set_crs(crs = 'EPSG:4326',epsg=27700, inplace=True, allow_override=True)

,CompanyName,geometry
0,!BIG IMPACT GRAPHICS LIMITED,POINT (-0.07869 51.52702)
1,"""K"" LINE (EUROPE) LIMITED",POINT (-0.09736 51.51757)
2,"""K"" LINE BULK SHIPPING (UK) LIMITED",POINT (-0.09736 51.51757)
3,"""K"" LINE HOLDING (EUROPE) LIMITED",POINT (-0.09736 51.51757)
4,"""K"" LINE LNG SHIPPING (UK) LIMITED",POINT (-0.09736 51.51757)
...,...,...
97964,ÉCLAT CENTURY SERVICES LIMITED,POINT (-0.08881 51.52725)
97965,ÉLEVAGE JOLIVET LTD,POINT (-0.08881 51.52725)
97966,ÜVIEW LIMITED,POINT (-0.08371 51.52559)
97967,‘ADORE BOURNEMOUTH LTD,POINT (-0.08013 51.52639)


# Spatial join

In [25]:
# get district
tech_roundabout_full = gpd.sjoin(tech_roundabout_coordinates, uk, 
                                 how = 'left', op = 'within').drop(columns = 'index_right')

# remove data points that are not in the uk --- bug when request for coordinate
tech_roundabout_full = tech_roundabout_full[~tech_roundabout_full.district.isna()]

# remove irrelevant data point --- postcode provided cannot be found on Google Map
tech_roundabout_full = tech_roundabout_full[~tech_roundabout_full['district'].isin(['Brentwood', 'Sheffield'])]

# Clean company data

In [26]:
tech_roundabout_district = uk[uk['district'].isin(tech_roundabout_full['district'].unique())]

In [27]:
df_tech = pd.merge(tech_industries , tech_roundabout, on='CompanyName', how='inner')
df_tech = pd.merge(df_tech, tech_roundabout_full, on='CompanyName', how='inner')
df_tech.head(3)
df_tech  = df_tech.drop(columns = [ 'IncorporationDate','month', 'CompanyNumber', 'sector_code_1', 'count'])
df_tech = df_tech.drop_duplicates()

df_tech_gdf = GeoDataFrame(df_tech)
df_tech_gdf['sector_name_1'] = df_tech_gdf['sector_name_1'].str.strip()
df_tech_gdf.head()
# 'year','IncorporationDate',  'CompanyStatus'

,year,CompanyName,CompanyStatus,sector_name_1,Industry,RegAddress.PostCode,RegAddress.PostTown,latitude,longitude,geometry,district
0,1991,MACXPERTS LIMITED,Active,Info Tech Consulting,informatin Tech,EC1Y 4SE,LONDON,51.521022,-0.090913,POINT (-0.09091 51.52102),Islington
1,1991,PREMIER TRAINING LIMITED,Active,Info Tech Consulting,informatin Tech,EC1V 2NX,LONDON,51.527246,-0.088808,POINT (-0.08881 51.52725),Islington
2,1991,INFIELD SYSTEMS LIMITED,Active,Data Companies,informatin Tech,EC2A 2EW,LONDON,51.521060,-0.080057,POINT (-0.08006 51.52106),City of London
3,1991,A C F FINANCIAL TECHNOLOGY LIMITED,Active,Software Dev.,informatin Tech,EC2V 8EH,LONDON,51.514624,-0.092348,POINT (-0.09235 51.51462),City of London
4,1991,CAPITA LAND LIMITED,Active,Software Dev.,informatin Tech,EC2V 7NQ,LONDON,51.515626,-0.093661,POINT (-0.09366 51.51563),City of London


## Add new column for colors

In [84]:
def district(counter):
    if counter['sector_name_1'] == 'Biotech Rsrch.':
        return 'darkgreen'
    elif counter['sector_name_1'] == 'Business Software Dev.':
        return 'lightgreen'
    elif counter['sector_name_1'] == 'Computer Game Companies':
        return 'lightblue'
    elif counter['sector_name_1'] == 'Computer facilities mgmt.':
        return 'gray'
    elif counter['sector_name_1'] == 'Data Companies':
        return 'darkpurple'
    elif counter['sector_name_1'] == 'Fund management activities':
        return 'orange'
    elif counter['sector_name_1'] == 'Info Tech Consulting':
        return 'lightred'
    elif counter['sector_name_1'] == 'Info Tech services':
        return 'cadetblue'
    elif counter['sector_name_1'] == 'Natural Sciences Rsrch.':
        return 'beige'
    elif counter['sector_name_1'] == 'Open-ended Investors':
        return 'purple'
    elif counter['sector_name_1'] == 'Satellite Telecommuncations':
        return 'green'
    elif counter['sector_name_1'] == 'Social Sciences Rsrch.':
        return 'pink'
    elif counter['sector_name_1'] == 'Software Dev.':
        return 'lightgray' 
    elif counter['sector_name_1'] == 'Technical testing and analysis':
        return 'green'
    elif counter['sector_name_1'] == 'Venture Capital':
        return 'blue'
    elif counter['sector_name_1'] == 'Web portals':
        return 'darkblue'
    elif counter['sector_name_1'] == 'Wireless Telecommuncations':
         return 'orange'
    else:
        return 'black'

df_tech_gdf["color"] = df_tech_gdf.apply(district, axis=1)
df_tech_gdf.head(3)

,year,CompanyName,CompanyStatus,sector_name_1,Industry,RegAddress.PostCode,RegAddress.PostTown,latitude,longitude,geometry,district,color,icons
0,1991,MACXPERTS LIMITED,Active,Info Tech Consulting,informatin Tech,EC1Y 4SE,LONDON,51.521,-0.0909131,POINT (-0.09091 51.52102),Islington,lightred,server
1,1991,PREMIER TRAINING LIMITED,Active,Info Tech Consulting,informatin Tech,EC1V 2NX,LONDON,51.5272,-0.0888078,POINT (-0.08881 51.52725),Islington,lightred,server
2,1991,INFIELD SYSTEMS LIMITED,Active,Data Companies,informatin Tech,EC2A 2EW,LONDON,51.5211,-0.0800573,POINT (-0.08006 51.52106),City of London,darkpurple,table


## Add new column for icons

In [85]:
def icon(counter):
    if counter['sector_name_1'] == 'Biotech Rsrch.':
        return 'leaf'
    elif counter['sector_name_1'] == 'Business Software Dev.':
        return 'wrench'
    elif counter['sector_name_1'] == 'Computer Game Companies':
        return 'gamepad'
    elif counter['sector_name_1'] == 'Computer facilities mgmt.':
        return 'laptop'
    elif counter['sector_name_1'] == 'Data Companies':
        return 'table'
    elif counter['sector_name_1'] == 'Fund management activities':
        return 'building'
    elif counter['sector_name_1'] == 'Info Tech Consulting':
        return 'server'
    elif counter['sector_name_1'] == 'Info Tech services':
        return 'signal'
    elif counter['sector_name_1'] == 'Natural Sciences Rsrch.':
        return 'pagelines'
    elif counter['sector_name_1'] == 'Open-ended Investors':
        return 'suitcase'
    elif counter['sector_name_1'] == 'Satellite Telecommuncations':
        return 'bars'
    elif counter['sector_name_1'] == 'Social Sciences Rsrch.':
        return 'users'
    elif counter['sector_name_1'] == 'Software Dev.':
        return 'code' 
    elif counter['sector_name_1'] == 'Technical testing and analysis':
        return 'comment'
    elif counter['sector_name_1'] == 'Venture Capital':
        return 'university'
    elif counter['sector_name_1'] == 'Web portals':
        return 'internet-explorer'
    elif counter['sector_name_1'] == 'Wireless Telecommuncations':
         return 'wifi'
    else:
        return 'chart-line'

df_tech_gdf["icons"] = df_tech_gdf.apply(icon, axis=1)
df_tech_gdf.head(3)

,year,CompanyName,CompanyStatus,sector_name_1,Industry,RegAddress.PostCode,RegAddress.PostTown,latitude,longitude,geometry,district,color,icons
0,1991,MACXPERTS LIMITED,Active,Info Tech Consulting,informatin Tech,EC1Y 4SE,LONDON,51.521,-0.0909131,POINT (-0.09091 51.52102),Islington,lightred,server
1,1991,PREMIER TRAINING LIMITED,Active,Info Tech Consulting,informatin Tech,EC1V 2NX,LONDON,51.5272,-0.0888078,POINT (-0.08881 51.52725),Islington,lightred,server
2,1991,INFIELD SYSTEMS LIMITED,Active,Data Companies,informatin Tech,EC2A 2EW,LONDON,51.5211,-0.0800573,POINT (-0.08006 51.52106),City of London,darkpurple,table


## Visualisation through Folium

In [88]:
from folium.plugins import TimestampedGeoJson

#set up the map
m = folium.Map(location=[51.51757, -0.08881], zoom_start=20, min_zoom=10, prefer_canvas=True, control_scale=True)

#set up the values
long=df_tech_gdf.longitude
lati=df_tech_gdf.latitude
coname=df_tech_gdf['CompanyName']
color=df_tech_gdf['color']
se=df_tech_gdf.sector_name_1
icon= df_tech_gdf['icons']

#adding different layers for the checkbox
feature_1 = folium.FeatureGroup(name='Biotech Rsrch.')
feature_2 = folium.FeatureGroup(name='Business Software Dev.')
feature_3 = folium.FeatureGroup(name='Computer Game Companies')
feature_4 = folium.FeatureGroup(name='Computer facilities mgmt.')
feature_5 = folium.FeatureGroup(name='Data Companies')
feature_6 = folium.FeatureGroup(name='Fund management activities')
feature_7 = folium.FeatureGroup(name='Info Tech Consulting')
feature_8 = folium.FeatureGroup(name='Info Tech services')
feature_9 = folium.FeatureGroup(name='Natural Sciences Rsrch.')
feature_10 = folium.FeatureGroup(name='Open-ended Investors')
feature_11 = folium.FeatureGroup(name='Satellite Telecommuncations')
feature_12 = folium.FeatureGroup(name='Social Sciences Rsrch.')
feature_13 = folium.FeatureGroup(name='Software Dev.')
feature_14 = folium.FeatureGroup(name='Technical testing and analysis')
feature_15 = folium.FeatureGroup(name='Venture Capital')
feature_16 = folium.FeatureGroup(name='Web portals')
feature_17 = folium.FeatureGroup(name='Wireless Telecommuncations')

for lat, lng, name, col,se,name,ico in zip(lati,long,coname, color,se,coname,icon):
    if se == 'Biotech Rsrch.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_1)
    elif se == 'Business Software Dev.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_2)
    elif se == 'Computer Game Companies':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_3)
    elif se == 'Computer facilities mgmt.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_4)
    elif se == 'Data Companies':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_5)
    elif se == 'Fund management activities':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_6)
    elif se == 'Info Tech Consulting':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_7)
    elif se == 'Info Tech services':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_8)
    elif se == 'Natural Sciences Rsrch.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_9)
    elif se == 'Open-ended Investors':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_10)
    elif se == 'Satellite Telecommuncations':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_11)
    elif se == 'Social Sciences Rsrch.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_12)
    elif se == 'Software Dev.':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_13)
    elif se == 'Technical testing and analysis':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_14)
    elif se == 'Venture Capital':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_15)
    elif se == 'Web portals':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_16)
    elif se == 'Wireless Telecommuncations':
        folium.Marker(location=[lat,lng],
                            tooltip=name,
                            icon=folium.Icon(color= col, icon=ico, angle=0, prefix='fa')).add_to(feature_17)



feature_1.add_to(m)
feature_2.add_to(m)
feature_3.add_to(m)
feature_4.add_to(m)
feature_5.add_to(m)
feature_6.add_to(m)
feature_7.add_to(m)
feature_8.add_to(m)
feature_9.add_to(m)
feature_10.add_to(m)
feature_11.add_to(m)
feature_12.add_to(m)
feature_13.add_to(m)
feature_14.add_to(m)
feature_15.add_to(m)
feature_16.add_to(m)
feature_17.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

#saving map as html
m.save('geospatial.html')